# Modeling of West African Ebola outbreak data using basic SIR and SEIR compartmental models

**Author: Veronica Scerra**  
**Last Updated: 2025-10-24**

**Purpose:**
Explore a dataset in which the SIR/SEIR/basic compartmental models can fit the data well

**Inputs & Dependencies:**
- Code: `src/dataio/ebola.py`, `src/epimodels/sir_scipy_fit.py`, `src/epimodels/seir_scipy_fit.py`, `src/epimodels/fitting/fitting.py`, `src/epimodels/utils/plotting.py`
- Libraries: `numpy`, `matplotlib`, `pandas`  
- Ebola data pulled from WHO/HDX

### Import paths and dependencies

In [1]:
import sys, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from pathlib import Path

from dataio.ebola import (
    EbolaPreprocessConfig,
    load_and_clean_ebola,
    list_available_indicators
)

import importlib
from epimodels.fitting.fitting_scipy import fit_sir_to_cumulative, fit_seir_to_cumulative
from epimodels.utils.plotting import plot_cumulative_fit
import epimodels.fitting.fitting_scipy as cal; importlib.reload(cal)
import epimodels.utils.plotting as viz; importlib.reload(viz)

%load_ext autoreload
%autoreload 2

PROJECT_ROOT = Path.cwd().parents[0]
DATA_PATH = PROJECT_ROOT/ "data" / "raw" / "ebola_data_db_format.csv"
SAVE_PATH = PROJECT_ROOT/ "data" / "processed" 

In [5]:
cfg = EbolaPreprocessConfig(
    country="Sierra Leone",
    case_definition="confirmed",  # or "confirmed_probable_suspected"
    resample="W",                 # "D" for daily, None for native cadence
    save_to=SAVE_PATH /"ebola_sierra_leone_weekly.csv"
)

ts = load_and_clean_ebola(DATA_PATH, cfg)
ts.head()

# set country population (Sierra Leone ~ 6.3M around 2014-2015)
N = 6.3e6
df = ts.copy()
dates = pd.to_datetime(df["date"])

### Fit SIR

In [6]:
sir_fit = cal.fit_sir_to_cumulative(df, N, I0_guess=20, beta_guess=0.25, gamma_guess=0.17, p_report_guess=0.8)

print(f"SIR: R0 = {sir_fit.R0_basic:.2f}, beta = {sir_fit.beta:.3f}, gamma = {sir_fit.gamma:.3f}, p = {sir_fit.p_report:.2f}, I0={sir_fit.I0:.1f}")
viz.plot_cumulative_fit(dates, df["cumulative_cases"].values, sir_fit.C_hat, "SIR fit - Sierra Leone")

KeyError: -1